In [1]:
import pandas as pd
import numpy as np

In [5]:
#tad_chip_strength = pd.read_csv('TAD_ChIP_strength.tsv',sep='\t')
tads = pd.read_csv('merge_TADs.bed',sep='\t',dtype={'chrom':str,'start':np.int32,'end':np.int32})
expression = pd.read_csv('sorted_expression.tsv',sep='\t',dtype={'chrom':str,'start':np.int32,'end':np.int32,'mean':np.float64})

In [6]:
expression

chrom     start       end   mean
0      chr1   3205901   3671498  0.155
1      chr1   3365731   3368549  0.005
2      chr1   3375556   3377788  0.045
3      chr1   3464977   3467285  0.015
4      chr1   3512451   3514507  0.010
...     ...       ...       ...    ...
28504  chrY  41619626  41644054  0.065
28505  chrY  58427109  58429470  0.020
28506  chrY  61317789  61343815  0.225
28507  chrY  84872139  84875484  0.005
28508  chrY  90499047  90531645  0.080

[28509 rows x 4 columns]

In [7]:
chroms = ['chr'+str(i) for i in range(1,20)]+['chrX']
tad_fpkm_chrs = []
for chrom in chroms:
    TADs_chr = tads.loc[tads.chrom==chrom].reset_index().drop(['index'],axis=1)
    expression_chr = expression.loc[expression.chrom==chrom].reset_index().drop(['index'],axis=1)
    fpkm_tad_means = []
    for idx, (c,s,e) in TADs_chr.iterrows():
        #print((s<=ChIPs_chr.start) & (ChIPs_chr.start<=s+40000))
        fpkm_tad_up = expression_chr.loc[expression_chr['start'].between(s,s+40000, inclusive = True) | expression_chr['end'].between(s,s+40000, inclusive = True),[False]*3+[True]]
        fpkm_tad_down = expression_chr.loc[expression_chr['start'].between(e,e+40000, inclusive = True) | expression_chr['end'].between(e,e+40000, inclusive = True),[False]*3+[True]]
        #print(ChIPs_chr.iloc[:,[True]+[False]*1,0])
        fpkm_tad_up_mean = fpkm_tad_up.mean(axis=0)
        fpkm_tad_down_mean = fpkm_tad_down.mean(axis=0)
        fpkm_tad_value = pd.DataFrame([fpkm_tad_up_mean.combine(fpkm_tad_down_mean, max, 0)])
        fpkm_tad_means.append(fpkm_tad_value)
    tmp = pd.concat(fpkm_tad_means,ignore_index=True,axis=0)
    tmp_chr = TADs_chr.join(tmp,how='inner')
    tad_fpkm_chrs.append(tmp_chr)
tad_fpkm = pd.concat(tad_fpkm_chrs,ignore_index=True,axis=0)
tad_fpkm.columns = ['chrom','start','end','border_fpkm']
#tad_chip_strength['border_fpkm']=tad_fpkm['border_fpkm']

In [8]:
tad_fpkm = tad_chip_strength.fillna(0)

NameError: name 'tad_chip_strength' is not defined

In [14]:
tad_chip_strength.to_csv('TAD_ChIP_strength_fpkm.tsv',sep='\t',index=False,quoting=3)

In [15]:
tad_fpkm = tad_chip_fpkm.assign(size=tad_fpkm['end']-tad_fpkm['start'] )

In [16]:
tad_fpkm.to_csv('TAD_borfer_fpkm_size.tsv',sep='\t',index=False,quoting=3)